## Init Config

In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from qids_package.qids import *
import warnings

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

warnings.filterwarnings("ignore")
seed = 257248

In [11]:
def train_valid_split(data_set, valid_ratio=0.2, seed=257248):
    '''Split provided training data into training set and validation set'''
    valid_set_size = int(valid_ratio * len(data_set))
    train_set, valid_set = train_test_split(data_set, test_size=valid_set_size, random_state=seed)
    return train_set, valid_set

## Load Data

In [2]:
train_fun = pd.read_csv("../data/first_round_train_fundamental_data.csv")
train_mar = pd.read_csv("../data/first_round_train_market_data.csv")
train_ret = pd.read_csv("../data/first_round_train_return_data.csv")

In [5]:
test_fun = pd.read_csv("./qids_package/first_round_test_fundamental_data.csv")
test_mar = pd.read_csv("./qids_package/first_round_test_market_data.csv")

In [14]:
train_fun, valid_fun = train_valid_split(train_fun)
train_mar, valid_mar = train_valid_split(train_mar)
train_ret, valid_ret = train_valid_split(train_ret)